In [1]:
import codecs
import numpy as np

In [2]:
train_ch = codecs.open("./train.conll").readlines()
dev_ch = codecs.open("./dev.conll").readlines()
test_ch = codecs.open("./final_test.txt").readlines()

In [3]:
print(test_ch[:5])
print(train_ch[:20])

['1\x01朝阳区小关北里000-0号\n', '2\x01朝阳区惠新东街00号\n', '3\x01朝阳区南磨房路与西大望路交口东南角\n', '4\x01朝阳区潘家园南里00号\n', '5\x01朝阳区向军南里二巷0号附近\n']
['浙 B-prov\n', '江 E-prov\n', '杭 B-city\n', '州 I-city\n', '市 E-city\n', '江 B-district\n', '干 I-district\n', '区 E-district\n', '九 B-town\n', '堡 I-town\n', '镇 E-town\n', '三 B-community\n', '村 I-community\n', '村 E-community\n', '一 B-poi\n', '区 E-poi\n', '\n', '浙 B-prov\n', '江 I-prov\n', '省 E-prov\n']


In [4]:
# 拼接训练集和验证集,并建立字典
ix_to_word = []
def ch2seq(data_ch):
    now_seq,now_label = '',[]
    data_seq,data_label = [],[]
    for ch in data_ch:
        if ' ' in ch:
            word,tag = ch.strip().split(' ')
            now_seq += word
            now_label.append(tag)
            if word not in ix_to_word:
                ix_to_word.append(word)
        else:
            # 地址序列的结尾
            data_seq.append(now_seq)
            data_label.append(now_label)
            now_seq,now_label = '',[]
    if now_seq:
        data_seq.append(now_seq)
        data_label.append(now_label)
    print("word_to_ix词典长度为{}".format(len(ix_to_word)))
    return data_seq,data_label

In [5]:
train_seq,train_label = ch2seq(train_ch)
dev_seq,dev_label = ch2seq(dev_ch)
print(len(train_seq))
print(len(dev_seq))

word_to_ix词典长度为2307
word_to_ix词典长度为2436
8856
1970


In [7]:
print(train_seq[:5])
print(train_label[:5])

['浙江杭州市江干区九堡镇三村村一区', '浙江省温州市平阳县海西镇宋埠公园南路0000号', '浙江省余姚市模具城金型路000号_样样红0A打印', '浙江省杭州市江干区白杨街道下沙开发区世茂江滨花园峻景湾00幢', '秋菱路浙江兰溪金立达框业有限公司']
[['B-prov', 'E-prov', 'B-city', 'I-city', 'E-city', 'B-district', 'I-district', 'E-district', 'B-town', 'I-town', 'E-town', 'B-community', 'I-community', 'E-community', 'B-poi', 'E-poi'], ['B-prov', 'I-prov', 'E-prov', 'B-city', 'I-city', 'E-city', 'B-district', 'I-district', 'E-district', 'B-town', 'I-town', 'E-town', 'B-poi', 'I-poi', 'I-poi', 'E-poi', 'B-road', 'E-road', 'B-roadno', 'I-roadno', 'I-roadno', 'I-roadno', 'E-roadno'], ['B-prov', 'I-prov', 'E-prov', 'B-district', 'I-district', 'E-district', 'B-poi', 'I-poi', 'E-poi', 'B-road', 'I-road', 'E-road', 'B-roadno', 'I-roadno', 'I-roadno', 'E-roadno', 'O', 'B-subpoi', 'I-subpoi', 'I-subpoi', 'I-subpoi', 'I-subpoi', 'I-subpoi', 'E-subpoi'], ['B-prov', 'I-prov', 'E-prov', 'B-city', 'I-city', 'E-city', 'B-district', 'I-district', 'E-district', 'B-town', 'I-town', 'I-town', 'E-town', 'B-devzone', 'I-devzo

In [6]:
from scipy.stats import describe
print(describe([
    len(x) for x in train_seq
]))
print(describe([
    len(x) for x in dev_seq
]))

DescribeResult(nobs=8856, minmax=(3, 69), mean=17.157859078590786, variance=32.80534308392208, skewness=1.0429906237323388, kurtosis=2.472641491590407)
DescribeResult(nobs=1970, minmax=(4, 76), mean=16.844162436548224, variance=32.96148190351463, skewness=1.516605352983294, kurtosis=7.126069095234971)


In [9]:
# 将训练集标签和验证集标签进行拼接,sum函数也可以用于列表的展开，相当于子元素相加
label_set = sum(train_label,[])+sum(dev_label,[])
# 将标签进行去重
label_set = list(set(label_set))
# label_set.insert(0, 'O')
print(len(label_set))

57


In [13]:
#建立字符和标签的索引词典
import os
import json
vocabs = {"word_vocab":{"pad_token": "<pad>","unk_token": "<unk>","mutable": False},"tag_vocab":{"pad_token":None,"unk_token": None,"mutable": False}}
vocabs["word_vocab"]["idx_to_token"] = ix_to_word
vocabs["tag_vocab"]["idx_to_token"] = label_set
file_path = './vocabs.json'
def build_vocabs(file_path):
    if os.path.exists(file_path):
        print("已经建立过词典了")
    else:
        with open(file_path,'w') as file:
            json.dump(vocabs,file,indent=4,ensure_ascii=False
                      )
            print("建立了新的索引词典")
build_vocabs(file_path)

建立了新的索引词典


In [14]:
from hanlp.utils.io_util import read_tsv
count = 0
for sent in read_tsv("./train.conll"):
    if count<5:
        print(sent)
    count += 1

[['浙', 'B-prov'], ['江', 'E-prov'], ['杭', 'B-city'], ['州', 'I-city'], ['市', 'E-city'], ['江', 'B-district'], ['干', 'I-district'], ['区', 'E-district'], ['九', 'B-town'], ['堡', 'I-town'], ['镇', 'E-town'], ['三', 'B-community'], ['村', 'I-community'], ['村', 'E-community'], ['一', 'B-poi'], ['区', 'E-poi']]
[['浙', 'B-prov'], ['江', 'I-prov'], ['省', 'E-prov'], ['温', 'B-city'], ['州', 'I-city'], ['市', 'E-city'], ['平', 'B-district'], ['阳', 'I-district'], ['县', 'E-district'], ['海', 'B-town'], ['西', 'I-town'], ['镇', 'E-town'], ['宋', 'B-poi'], ['埠', 'I-poi'], ['公', 'I-poi'], ['园', 'E-poi'], ['南', 'B-road'], ['路', 'E-road'], ['0', 'B-roadno'], ['0', 'I-roadno'], ['0', 'I-roadno'], ['0', 'I-roadno'], ['号', 'E-roadno']]
[['浙', 'B-prov'], ['江', 'I-prov'], ['省', 'E-prov'], ['余', 'B-district'], ['姚', 'I-district'], ['市', 'E-district'], ['模', 'B-poi'], ['具', 'I-poi'], ['城', 'E-poi'], ['金', 'B-road'], ['型', 'I-road'], ['路', 'E-road'], ['0', 'B-roadno'], ['0', 'I-roadno'], ['0', 'I-roadno'], ['号', 'E-roadno'], ['